In [118]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra
using Pkg
using IJulia

In [119]:
include("../Test Models/COVIDResourceAllocation.jl")
using .COVIDResourceAllocation

In [120]:
pct_nurses_available = 0.5
patients_per_nurse_covid = 2.5
nurse_hrs_per_week_covid = 36

@show nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7
@show nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid

@show nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid;

nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7 = 5.142857142857143
nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid = 4.666666666666666
nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid = 1.8666666666666665


In [121]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 4, 1)
end_date   = Date(2020, 5, 1)
travel_threshold_hours = 4.0
pct_beds_available = 0.25
travel_threshold_hours = 4.0
hospitalized_days = 14;

In [122]:
N = length(states);
T = (end_date - start_date).value + 1;

In [123]:
forecast_active = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
);

In [ ]:
forecast_admitted = forecast(
    states, start_date,  end_date,
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
);

In [ ]:
forecast_initial = forecast(
    states, start_date-Dates.Day(1), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
)[:];

In [ ]:
forecast_discharged = forecast(
    states, start_date-Dates.Day(hospitalized_days), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = hcat(forecast_discharged, zeros(Float32, N, T - hospitalized_days));

In [ ]:
beds = n_beds(states, bed_type=:all, pct_beds_available=pct_beds_available);
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours);

In [ ]:
demand = forecast_active * Float32(nurse_days_per_patient_day_covid);

In [ ]:
nurses = n_nurses(states) * Float32(pct_nurses_available);

In [ ]:
t  = 1
N = 11
contaminated_hospitals = [0,0,0,0,0,0,1,1,0,0,0]
stage1_matrix = falses(N,N)

    for i in 1:N  
        for j in 1:N 
            # check that the edge is directed toward the contaminated hospital
            # if true set the matrix idx to 1 else leave it as 0 
            stage1_matrix[i,j] = (contaminated_hospitals[j] == 1)
        end
    end
    
stage2_matrix = falses(N,N)
for i in 1:N  
    for j in 1:N 
        # Manditorily make a fully connected graph among non-contaminated hospitals
        stage2_matrix = ((contaminated_hospitals[i] == 0) && contaminated_hospitals[j] == 0)
    end
end

model1 = reusable_resource_allocation(
        nurses,
        zeros(Float32, size(demand[:, t:t+1])...),
        demand[:, t:t+1],
        stage1_matrix,
        obj_dir = :shortage,  
        send_new_only = false,
        sendrecieve_switch_time = 0,
        min_send_amt = 0,
        smoothness_penalty = 0,
        setup_cost = 0,
        sent_penalty = 0,
        verbose = true
    )
    
    sent_nurses_contam = value.(model1[:sent])
    sent_nurses_contam_matrix = sum(sent_nurses_contam, dims = 3)
    sent_nurses_contam_net_array = zeros(11)  # Ensure this dimension matches the number of hospitals
    for i in 1:11
        sent_nurses_contam_net_array[i] = sum(sent_nurses_contam_matrix[i, :]) - sum(sent_nurses_contam_matrix[:, i])
    end

    nurses_afterstage1 = nurses+sent_nurses_contam_net_array

model2 = patient_nurse_allocation(
        beds,
        forecast_initial[:,t:t+1],
        forecast_discharged[:, t:t+1],
        forecast_admitted[:, t:t+1],
        nurses_afterstage1,
        stage2_matrix,
        los=11,
        nurse_days_per_patient_day=2.0,
        smoothness_penalty=0,
        setup_cost=0,
        sent_penalty=0,
        balancing_thresh_patients=1.0,
        balancing_penalty_patients=0,
        nurse_target_load=1.25,
        nurse_target_load_gap=0.25,
        nurse_load_penalty=0,
        disallow_nurse_shortage_sent=false,
        disallow_nurse_shortage_newpatients=false,
        severity_weighting=false,
        no_artificial_overflow=false,
        no_artificial_shortage=false,
        verbose =false,
    )

sent_nurses_uncontam = value.(model2[:sentnurses])
sent_patients_uncontam = value.(model2[:sentpatient])
        

In [ ]:
contaminated_hospitals = [0,0,0,0,0,0,1,1,0,0,0]
stage1_matrix = falses(N,N)

    for i in 1:N  
        for j in 1:N 
            # check that the edge is directed toward the contaminated hospital
            # if true set the matrix idx to 1 else leave it as 0 
            stage1_matrix[i,j] = (contaminated_hospitals[j] == 1)
        end
    end
    
stage2_matrix = falses(N,N)
for i in 1:N
    for j in 1:N
        # Mandatorily make a fully connected graph among non-contaminated hospitals
        # Only set to true if both hospitals i and j are not contaminated
        stage2_matrix[i, j] = (contaminated_hospitals[i] == 0) && (contaminated_hospitals[j] == 0)
    end
end



In [113]:
model2 = patient_nurse_allocation(
        beds,
        forecast_initial,
        forecast_discharged[:, t:t+1],
        forecast_admitted[:, t:t+1],
        nurses_afterstage1,
        stage2_matrix,
        los=11,
        nurse_days_per_patient_day=2.0,
        sendreceive_switch_time=0,
        min_send_amt = 0,
        smoothness_penalty=0,
        setup_cost=0,
        sent_penalty=0,
        balancing_thresh_patients=1.0,
        balancing_penalty_patients=0,
        nurse_target_load=1.25,
        nurse_target_load_gap=0.25,
        nurse_load_penalty=0,
        disallow_nurse_shortage_sent=false,
        disallow_nurse_shortage_newpatients=false,
        severity_weighting=false,
        no_artificial_overflow=false,
        no_artificial_shortage=false,
        verbose =false,
    )

ArgumentError: ArgumentError: invalid Array dimensions

In [43]:
function contagion_model( 
    adj_matrix::BitArray{2}, 
    contaminated_hospitals::Array{<:Real,1}, 
    initial_nurses::Array{<:Real,1}, 
    demand::,
    initial_patients,
    discharged_patients,
    admitted_patients
    T::Real=0,
)
    #= 
        adj_matrix [N by N matrix]: specifies the edges between hospitals in a network; N is the number 
                    of hospitals in the network

        contaminated hospitals [array of size N]: an array where index i is 1 if the hospital is contaminated 
                                0 otherwise

        initial_capacities [array of size N]: an array where index i is the number of initial capacities in hospital i 
                            
        patients [N by 2 matrix]: initial patients at each hospital on day 1 and day 2

        T [int]: the time horizon
    =#

    # used to create new graph of the exact same size 
    N, T = size(demand)

    # construct the Stage 1 graph 

    stage1_matrix = falses(N,N)
    
    for i in 1:N  
        for j in 1:N 
            # check that the edge is directed toward the contaminated hospital
            # if true set the matrix idx to 1 else leave it as 0 
            stage1_matrix[i,j] = (contaminated_hospitals[j] == 1)
        end
    end

    # construct the Stage 2 graph 
    stage2_matrix = falses(N,N)
    for i in 1:N  
        for j in 1:N 
            # Manditorily make a fully connected graph among non-contaminated hospitals
            stage2_matrix = ((contaminated_hospitals[i] == 0) && contaminated_hospitals[j] == 0)
        end
    end

    for t in 1:T
        ###########
        # STAGE 1 #
        ###########
        # In stage 1 we only run resource redistribution model on a modified network.
        # The modified network is as follows: the only edges that exist are those that are directed to the contaminated
        # hosptials. In other words, we can only send resources to the contaminated hospitals in this stage. 
        # To accomplish this, we run a resource redistribtuion model. 
        
        # t is the step size of days we want to forward after each round

        model1 = reusable_resource_allocation(
            initial_nurses,
            zeros(Float32, size(demand[:, t:t+1])...),
            demand[:, t:t+1],
            stage1_matrix,
            obj_dir = :shortage,  
            send_new_only = false,
            sendrecieve_switch_time = 0,
            min_send_amt = 0,
            smoothness_penalty = 0,
            setup_cost = 0,
            sent_penalty = 0,
            verbose = true
        )
        
        sent_nurses_contam = value.(model1[:sent])
        sent_nurses_contam_matrix = sum(sent_nurses_contam, dims = 3)
        sent_nurses_contam_net_array = zeros(N)  # Ensure this dimension matches the number of hospitals
        for i in 1:N
            sent_nurses_contam_net_array[i] = sum(sent_nurses_contam_matrix[i, :]) - sum(sent_nurses_contam_matrix[:, i])
        end

        nurses_afterstage1 = initial_nurses + sent_nurses_contam_net_array
        
        ###########
        # STAGE 2 #
        ###########
        # In stage 2 we run a mixing patient and resource model on a modified network.
        # The modified network is as follows: no nodes are directed to the contaminated hosptials.
        # Uncontaminated hospitals are interconnected to each other. In other words, the goal is to balance the loads
        # between uncontaminated hospitals in this stage after helping the contaminated hospitals.
        # Note that the initial_nurses are initial_nurses before stage 1 - the number of nurses
        # sent to the contaminated_hospitals.
        # To accomplish this, we run a patient_nurse_allocation model.
        
        model2 = patient_nurse_allocation(
            beds = beds,
            initial_patients = initial_patients[:, t:t+1],
            discharged_patients = discharged_patients[:, t:t+1],
            admitted_patients = admitted_patients[:, t:t+1],
            initial_nurses = nurses_afterstage1,
            adj_matrix = stage2_matrix;
            los=11,
            nurse_days_per_patient_day=2.0,
            smoothness_penalty=0,
            setup_cost=0,
            sent_penalty=0,
            balancing_thresh_patients=1.0,
            balancing_penalty_patients=0,
            nurse_target_load=1.25,
            nurse_target_load_gap=0.25,
            nurse_load_penalty=0,
            disallow_nurse_shortage_sent=false,
            disallow_nurse_shortage_newpatients=false,
            severity_weighting=false,
            no_artificial_overflow=false,
            no_artificial_shortage=false,
            verbose =false,
        )

        sent_nurses_uncontam = value.(model2[:sentnurses])
        sent_patients_uncontam = value.(model2[:sentpatient])
        
        # initial value for the next round
        nurses = nurses_afterstage1 - sent_nurses_uncontam
        demand = demand - sent_patients_uncontam
        
        
        
        t = t+1

    end 
end

contagion_model (generic function with 2 methods)

In [ ]:
Using

In [23]:
import XLSX
xf = XLSX.readxlsx("CaseCountData.xlsx")
sheetnames = XLSX.sheetnames(xf)
xf["Trends"]

9-element Vector{String}:
 "Case and Fatalities_ALL"
 "Case and Fatalities_2020"
 "Case and Fatalities_2021"
 "Case and Fatalities_2022"
 "Case and Fatalities_2023"
 "Trends"
 "Molecular Positivity Rate"
 "Hospitalizations"
 "Hospitalization by Day"

In [26]:
using DataFrames, XLSX
df = DataFrame(XLSX.readtable("CaseCountData.xlsx", "Trends"))

Row,Date,Cumulative Confirmed Cases,New Confirmed Cases,Cumulative Probable Cases,New Probable Cases,Cumulative Fatalities,Total Fatalities Current Day
,Any,Any,Any,Any,Any,Any,Any
1,2020/03/04,0,0,0,0,missing,missing
2,2020/03/05,0,0,0,0,missing,missing
3,2020/03/06,1,1,0,0,missing,missing
4,2020/03/07,1,0,0,0,0,missing
5,2020/03/08,3,2,1,1,0,missing
6,2020/03/09,9,6,1,0,0,missing
7,2020/03/10,19,10,2,1,0,0
8,2020/03/11,25,6,3,1,0,0
9,2020/03/12,30,5,3,0,0,0
